### EDA Automation Notebook
This notebook is a jupyter notebook template for all my EDA projects. This will contain various checks that I commonly use in my projects. Since this is an automation or a template, this does not contain any test data. It will contain the following checks as seen below:
- Step 1: Loading the data
- Step 2: Checking the basic data information
- Step 3: Data quality assessment
- Step 4: Summary statistics
- Step 5: Correlations and outliers
- Step 6: Answering questions

There will be another python file included in this notebook and this prints out the pdf report instead of the notebook as this is used for data exploration only and not for answering questions.

In [ ]:
#Import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

##### Step 1: Loading the data

##### Step 2: Checking the basic data information

##### Step 3: Data quality assessment

##### Step 4: Summary statistics

##### Step 5: Correlations and outliers

##### Step 6: Answering questions